# ĐỒ ÁN MÔN HỌC
## Nhập môn khoa học dữ liệu - CQ2022/21 - Nhóm 2
### Giáo viên hướng dẫn: Thầy Lê Nhựt Nam
### Thành viên nhóm:
    21120201	Bùi Đình Bảo
    22120017	Trương Tiến Anh
    22120023	Lê Nguyễn Gia Bảo
    22120029	Nguyễn Hữu Bền
    22120043	Đoàn Minh Cường

## Giai đoạn: Thu thập dữ liệu

### Trang web thực hiện cào dữ liệu: `https://mobilecity.vn/dien-thoai`

### 1. Nhập thư viện:

In [ ]:
# Thư viện điều khiển trình duyệt web
from selenium import webdriver
# Dịch vụ điều khiển Chrome
from selenium.webdriver.chrome.service import Service
# Quản lý driver của Chrome
from webdriver_manager.chrome import ChromeDriverManager
# Định vị các phần tử trên trang web
from selenium.webdriver.common.by import By
# Xử lý các ngoại lệ
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
# Chờ đợi các điều kiện cụ thể trên trang web
from selenium.webdriver.support.ui import WebDriverWait
# Các điều kiện cụ thể
from selenium.webdriver.support import expected_conditions as EC
# Thư viện thời gian
import time
# Thư viện phân tích cú pháp HTML
from bs4 import BeautifulSoup
# Thư viện xử lý dữ liệu JSON
import json
# Thư viện phân tích cú pháp URL
from urllib.parse import urlparse
# Thư viện xử lý csv
import csv

### 2. Thu thập dữ liệu:

#### 2.1. Lấy mã nguồn trang web:
- Tạo một **instance** của trình duyệt.
    - **Instance** của trình duyệt là một đối tượng của trình duyệt web được tạo ra bởi `Selenium WebDriver`.
    - **Instance** cho phép điều khiển và tương tác với trình duyệt thông qua mã `Python`.
- Liên tục nhấn nút `Xem thêm` để tải thêm sản phẩm.
- Lấy mã nguồn của trang sau khi đã nhấn hết các nút `Xem thêm`.

In [2]:
# Tạo instance của trình duyệt
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Truy cập vào trang web
url = "https://mobilecity.vn/dien-thoai"
driver.get(url)

# Hàm nhấn nút "Xem thêm" nếu có thể
def click_see_more_button():
    try:
        # Tìm nút "Xem thêm điện thoại"
        see_more_button = driver.find_element(By.ID, "product_view_more")
        if see_more_button.is_displayed():
            # Nhấn nút
            see_more_button.click()
            # Đợi một chút để trang tải thêm sản phẩm
            time.sleep(2)
            return True
    except (NoSuchElementException, ElementClickInterceptedException):
        # Không tìm thấy nút hoặc không thể nhấn
        return False
    return False

# Liên tục nhấn nút "Xem thêm" cho đến khi không còn nút nào
while click_see_more_button():
    pass

# Lấy source page sau khi đã nhấn hết các nút "Xem thêm"
page_source = driver.page_source
# Đóng trình duyệt
driver.quit()

#### 2.2. Truy xuất danh sách các liên kết đến chi tiết sản phẩm của từng điện thoại:
- Phân tích cú pháp HTML của trang.
- Tìm tất cả các sản phẩm.
- Lưu các liên kết đến chi tiết sản phẩm.
- Sử dụng cấu trúc dữ liệu phù hợp để lưu trữ: `list` trong Python.

In [3]:
# Phân tích cú pháp HTML
soup = BeautifulSoup(page_source, "html.parser")

# Tìm tất cả các sản phẩm
product_items = soup.find_all("div", class_="product-item-info")

# Lưu các liên kết vào danh sách
links = []

for item in product_items:
    # Tìm thẻ <a> chứa liên kết đến chi tiết sản phẩm
    link_tag = item.find("a", href=True)
    if link_tag:
        # Lấy giá trị href
        link = link_tag['href']
        links.append(link)

In [4]:
def is_valid_url(url):
    """Check if the URL is valid."""
    parsed = urlparse(url)
    return bool(parsed.scheme and parsed.netloc)


#### 2.3. Thu thập dữ liệu chi tiết sản phẩm và lưu vào danh sách:
- Với mỗi liên kết, thử thu thập các thông tin về chi tiết sản phẩm cần thiết *(Tên sản phẩm, Giá mới,Giá cũ, Màu sắc, Các phiên bản bộ nhớ, Thời gian bảo hành, Thông số kỹ thuật, Đánh giá, Số lượt đánh giá và hỏi đáp, Đường dẫn)*.
- Xử lý khi gặp lỗi hoặc không có thông tin mong đợi.
- Sử dụng cấu trúc dữ liệu phù hợp để lưu trữ: `dictionary` trong Python.

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

product_data = []
counter = 0  

for link in links:
    link = link.strip()  # Xóa khoảng trắng đầu và cuối
    if not is_valid_url(link):
        print(f"Invalid URL: {link}")
        continue  # Skip invalid URLs

    try:
        driver.get(link)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.title")))
    except Exception as e:
        print(f"Error loading {link}: {e}")
        continue  # Skip this link and move to the next

    # Lấy HTML trang hiện tại
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Thu thập tên sản phẩm
    try:
        product_title = soup.find("h1", class_="title").text.strip()
    except AttributeError:
        product_title = "Không có thông tin"

    # Thu thập giá sản phẩm (giá mới và giá cũ)
    try:
        price_new = soup.find("p", class_="price").text.strip()
    except AttributeError:
        price_new = "Không có thông tin"

    try:
        price_old = soup.find("p", class_="price-old").text.strip()
    except AttributeError:
        price_old = "Không có thông tin"

    # Thu thập màu sắc
    try:
        color_items = soup.find_all("div", class_="color-item")
        colors = [color['data-title'] for color in color_items]
    except AttributeError:
        colors = ["Không có thông tin"]

    # Lấy thông tin về các phiên bản bộ nhớ
    try:
        storage_items = soup.find_all('div', class_='storage-item attribute-item')
        storage_options = [item.text.strip() for item in storage_items]
    except AttributeError:
        storage_options = ["Không có thông tin"]

    # Lấy thông tin về thời gian bảo hành
    try:
        warranty_text = soup.find('span', class_='warranty-content-default').get_text(strip=True)
    except AttributeError:
        warranty_text = "Không có thông tin"

    # Lấy thông số kỹ thuật
    try:
        specs_table = soup.find('div', class_='product-info-content').find('table')
        specs = {}
        if specs_table:
            rows = specs_table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    specs[cols[0].text.strip()] = cols[1].text.strip()
    except AttributeError:
        specs = {"Không có thông số kỹ thuật": "N/A"}

    # Thu thập thông tin đánh giá
    try:
        rating = soup.find('div', class_='comment-vote__star-number').get_text(strip=True)
    except AttributeError:
        rating = "Không có thông tin"

    # Thu thập thông tin số lượt đánh giá và hỏi đáp
    try:
        total_reviews = soup.find('div', class_='comment-vote__star-total').get_text(strip=True)
        total_reviews = total_reviews.replace("đánh giá và hỏi đáp", "").strip()
    except AttributeError:
        total_reviews = "Không có thông tin"

    product_data.append({
        "Tên sản phẩm": product_title,
        "Giá mới": price_new,
        "Giá cũ": price_old,
        "Màu sắc": ", ".join(colors),
        "Các phiên bản bộ nhớ": ", ".join(storage_options),
        "Thời gian bảo hành": warranty_text,
        "Thông số kỹ thuật": json.dumps(specs, ensure_ascii=False),
        "Đánh giá": rating,
        "Số lượt đánh giá và hỏi đáp": total_reviews,
        "Đường dẫn":link,
    })
    
    counter += 1
    print(f"\rĐã thu thập sản phẩm thứ {counter}", end='', flush=True)

# In thông báo khi hoàn thành
print("Hoàn thành!!!")  

Đã thu thập sản phẩm thứ 1420Hoàn thành!!!


#### 2.4. Lưu dữ liệu sản phẩm vào file CSV và đóng trình duyệt:

In [6]:
# Lưu dữ liệu vào file CSV
with open('data.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = product_data[0].keys()  # Lấy các tên trường từ phần tử đầu tiên
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in product_data:
        writer.writerow(data)

# Đóng trình duyệt
driver.quit()